In [1]:
import sys
import os
from pathlib import Path
import pickle
import torch
import pprint
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

In [2]:
args = parser.parse_args([])
args.task = 'normal'
args.metric = 'nominal'
args.SoftEva = True
args = FormulateArgs(args)

In [3]:
results = torch.zeros([13,10,4])

for seed in range(10):
    for ds in range(13):
        args.SEED = seed
        args.DATASET = ds

        valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset')
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset')

        modelname = f"model_pNN_data_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
        
        
        if os.path.isfile(f'./models/{modelname}'):                
            print(modelname)
            
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            model.SetParameter('args', args)

            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)

            # Validation phase
            total_val_samples = 0
            total_val_acc = 0.0
            total_val_power = 0.0
            with torch.no_grad():
                for x, y in valid_loader:
                    X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)

                    acc_valid_batch, power_valid_batch = evaluator(model, X_valid, y_valid)
                    batch_size = X_valid.size(0)

                    total_val_samples += batch_size
                    total_val_acc += acc_valid_batch * batch_size
                    total_val_power += power_valid_batch * batch_size

            acc_valid = total_val_acc / total_val_samples
            power_valid = total_val_power / total_val_samples

            # Test phase
            total_test_samples = 0
            total_test_acc = 0.0
            total_test_power = 0.0
            with torch.no_grad():
                for x, y in test_loader:
                    X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

                    acc_test_batch, power_test_batch = evaluator(model, X_test, y_test)
                    batch_size = X_test.size(0)

                    total_test_samples += batch_size
                    total_test_acc += acc_test_batch * batch_size
                    total_test_power += power_test_batch * batch_size

            acc_test = total_test_acc / total_test_samples
            power_test = total_test_power / total_test_samples

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
            results[ds,seed,2] = power_valid
            results[ds,seed,3] = power_test
            
        else:
            results[ds,seed,:] = float('nan')

model_pNN_data_acuteinflammation_seed_00.model
model_pNN_data_balancescale_seed_00.model
model_pNN_data_breastcancerwisc_seed_00.model
model_pNN_data_cardiotocography3clases_seed_00.model
model_pNN_data_energyy1_seed_00.model
model_pNN_data_energyy2_seed_00.model
model_pNN_data_iris_seed_00.model
model_pNN_data_mammographic_seed_00.model
model_pNN_data_pendigits_seed_00.model
model_pNN_data_seeds_seed_00.model
model_pNN_data_tictactoe_seed_00.model
model_pNN_data_vertebralcolumn2clases_seed_00.model
model_pNN_data_vertebralcolumn3clases_seed_00.model
model_pNN_data_acuteinflammation_seed_01.model
model_pNN_data_balancescale_seed_01.model
model_pNN_data_breastcancerwisc_seed_01.model
model_pNN_data_cardiotocography3clases_seed_01.model
model_pNN_data_energyy1_seed_01.model
model_pNN_data_energyy2_seed_01.model
model_pNN_data_iris_seed_01.model
model_pNN_data_mammographic_seed_01.model
model_pNN_data_pendigits_seed_01.model
model_pNN_data_seeds_seed_01.model
model_pNN_data_tictactoe_seed

In [4]:
K = 3  # for example

valid_acc_data = results[:, :, 0]

# Obtain the top K validation accuracies and their indices for each row
top_k_values, top_k_seeds = torch.topk(valid_acc_data, K, dim=1)

# Initialize the tensor to hold the best K results for each row
best_data_k = torch.zeros(valid_acc_data.size(0), K, results.size(2))

# Populate best_data_k using the top K seeds
for i in range(valid_acc_data.size(0)):
    for j in range(K):
        best_data_k[i, j] = results[i, top_k_seeds[i, j]]

results_mean = torch.mean(best_data_k, dim=1)
results_std = torch.std(best_data_k, dim=1)

final = torch.cat([results_mean, results_std], dim=1)

np.savetxt('./result.txt', final.detach().numpy(), fmt='%.9f', delimiter='\t')